# Linear vs nonlinear phase

In [ ]:
# Import necessary libraries
import numpy as np
import scipy.signal as sp
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as wg

plt.rcParams["figure.figsize"] = (12,3)

# Physical analogy: refraction

<div margin: 30px;"><img src="img/refraction.jpg" width="600"></div>

<div margin: 30px;"><img src="img/prism.jpg" width="600"></div>

## Square wave from oscillations

An antisymmetric, balanced, discrete-time square wave can be expressed as
$$
    \mathbf{q} = \frac{1}{N} \sum_{k=0}^{N-1} Q[k] \mathbf{w}_k 
$$
with
$$
    Q[k] = \begin{cases} 0 & k \text{~even} \\ \displaystyle \frac{-2j}{\tan(\pi k /N)}  & k \text{~odd} \end{cases}
$$


Let's see what happens if we mess with the phase of the DFT coefficients

In [ ]:
def wrap(x, r):
    d = 2 * r if np.isscalar(r) else np.abs(r[0] - r[1])
    s = 0 if np.isscalar(r) else (r[0] + r[1]) / 2
    return x - d * np.floor((x - s) / d + 0.5)

In [ ]:
class sqw:
    def __init__(self, N):
        assert N % 2 == 0, 'N must be even'
        self.N = N
        rpp = np.exp(2j * (np.random.rand(N // 2 - 1) - 0.5) * np.pi)
        self.rp = 2 * np.pi * (np.random.rand(N // 2) - 0.5)    #np.r_[0, rpp, 0, np.conj(rpp[::-1])]

    def show(self, L, delay=0, random_phase=False):
        Q = np.zeros(self.N, dtype=complex)
        phase_offset = delay * 2 * np.pi / self.N
        for m in range(L//2+1):
            k = 2 * m + 1
            Q[k] = 2 / np.tan(np.pi / self.N * k) * np.exp(1j * (-np.pi / 2 + phase_offset * k + (self.rp[k] if random_phase else 0)))
            Q[self.N-k] = np.conj(Q[k])
        q = np.fft.ifft(Q).real
        
        plt.figure(figsize=(16, 6));
        plt.subplot(1,2,1)
        plt.plot(q)
        plt.title(f"partial sum with {2*L+1} terms" + (f", linear phase factor {2 * delay / self.N :.2f} $\\pi$" if delay != 0 else ""))
        plt.grid()
        plt.ylim(-2,2)
        plt.subplot(2,2,2)
        plt.stem(np.abs(Q[:self.N]))
        plt.title(f"$|Q[k]|$")
        plt.subplot(2,2,4)
        plt.stem(wrap(np.angle(Q[:self.N]), np.pi))
        plt.ylim(-3.4,3.4)
        plt.title(f"$\\angle Q[k]$")
        plt.tight_layout()    

    def interact(self):
        return wg.interactive(self.show, L=wg.IntSlider(min=1, max=self.N//2-1, value=1, step=2), 
               delay=wg.IntSlider(min=-self.N//2, max=self.N//2, value=0, description='linear phase factor'))

In [ ]:
sqw(100).interact()

# Effects of linear phase on signal shape

The following functions can be used as a black box for now

In [ ]:
def magnitude_response(b, a, sf, points=None, color='C0'):
    L = (points or max(len(a), len(b))) // 2
    points = 2 * L + 1
    w = 2 * np.pi * np.arange(-L, L+1) / points
    A, B = np.zeros(points, dtype='complex'), np.zeros(points, dtype='complex')
    for n, bn in enumerate(b):
        B += bn * np.exp(-1j * n * w)
    for n, an in enumerate(a):
        A += an * np.exp(-1j * n * w)
    A, B = np.abs(A), np.abs(B)
    M = B / np.where(A == 0, 1e-20, A)
    f = w / np.pi * sf / 2
    plt.plot(f, M, color, lw=2)

In [ ]:
def SRRC(K, beta, L=6):
    # Type-I FIR truncation of a root-raised-cosine impulse response with
    #  cutoff frequency pi/K and rolloff beta. The impulse is truncated after
    #  L bauds (ie, the filter will have 2LK + 1 taps); usually L=6.
    N = int(L * K)
    r = np.zeros(2 * N + 1)
    for n in range(-N, N+1):   
        t = n / K  # from baud rate to sampling rate
        if n == 0:
            r[n+N] = 1.0 - beta + (4 * beta / np.pi)
        elif np.abs(n) == K / (4 * beta):
            r[n+N] = (beta / np.sqrt(2)) * (((1 + 2 / np.pi) * \
                     (np.sin(np.pi / (4 * beta)))) + ((1 - 2 / np.pi) * (np.cos(np.pi / (4 * beta)))))
        else:
            r[n+N] = (np.sin(np.pi * (1 - beta) * t) + 4 * beta * t * np.cos(np.pi * (1 + beta) * t)) / \
                    (np.pi * t * (1 - (4 * beta * t) * (4 * beta * t)))
    return r / np.sqrt(K)

this is a typical impulse used for transmitting information

In [ ]:
r = SRRC(10, 0.3)
plt.plot(r);

the pulse is nice bcecause it has a compact spectrum with controllable width

In [ ]:
magnitude_response(r, [1], sf=2*np.pi)

here is a simple transmitter, sending well-spaced positive and negative pulses

In [ ]:
def modulate(bits, K, spacing=None):
    spacing = spacing or 2 * K
    rc = SRRC(K, 0.3)
    M = len(rc)
    x = np.zeros(bits * spacing * K + M)
    for n in range(bits):
        ix = n * spacing * K
        x[ix:ix+M] += np.sign(np.random.randn()) * rc
    return x

In [ ]:
K = 10
x = modulate(10, K)

plt.plot(x);

the spectrum of the transmitted signal has the same shape of the individual pulse

In [ ]:
magnitude_response(x, [1], sf=2*np.pi, color='C0');

let's build a matched linear-phase lowpass filter. This can be used to remove out-of-band noise, for instance

In [ ]:
L = 150
M = 2 * L + 1
h = sp.remez(M, [0, 1/K, 1.3/K, 1], [1, 0], weight=[1, 1], Hz=2)

magnitude_response(x / 10, [1], sf=2*np.pi)
magnitude_response(h, [1], sf=2*np.pi, color='C1')

if we filter the transmitted signal, the shape of the pulses is preserved

In [ ]:
w = sp.lfilter(h, 1, x)[len(h)//2:]

plt.plot(x)
plt.plot(w);

now let's randomize the phase of the filter so it becomes nonlinear. This does not change the magnitude repsonse of the filter

In [ ]:
p = (np.random.rand(L) - 0.5) * 2 * np.pi
p = np.r_[ [0], p, -p[::-1] ]

hn = np.real(np.fft.ifft(np.abs(np.fft.fft(h)) * np.exp(1j * p)))

In [ ]:
magnitude_response(h, [1], sf=2*np.pi, color='C1')
magnitude_response(hn, [1], sf=2*np.pi, color='C3')

but look at the shape of the output

In [ ]:
v = sp.lfilter(hn, 1, x)[len(h)//2:]

plt.plot(x)
plt.plot(w)
plt.plot(v, 'C3');

In [ ]:
s = slice(400, 800)
plt.plot(x[s])
plt.plot(w[s])
plt.plot(v[s], 'C3');